In [1]:
pip install imutils

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25834 sha256=b70eb96975427891aed1bf93d48c1c56ca92549128ebc47e80503c931cc20c68
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


# **IMPORT CÁC THƯ VIỆN CẦN THIẾT**

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from imutils import paths
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras.utils import to_categorical
from sklearn.svm import SVC
from joblib import dump, load, Parallel, delayed

2024-06-13 07:43:33.203834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 07:43:33.203988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 07:43:33.371730: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **LOAD VÀ XỬ LÍ DỮ LIỆU**

In [3]:
train_dir = '/kaggle/input/fer2013/train'
test_dir = '/kaggle/input/fer2013/test'

In [4]:
# Danh sách cảm xúc
emotions = ['disgust', 'fear', 'happy', 'surprise']

In [5]:
# Hàm để phát hiện khuôn mặt
def detect_faces(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

In [6]:
# Hàm để trích xuất đặc trưng HOG từ khuôn mặt
def extract_hog_features(image):
    fd = hog(image, orientations=7, pixels_per_cell=(8, 8), cells_per_block=(4, 4), block_norm='L2-Hys', transform_sqrt=False)
    return fd

In [7]:
# Hàm để đọc dữ liệu từ thư mục
def load_data(directory):
    data = []
    labels = []
    for emotion in emotions:
        emotion_dir = os.path.join(directory, emotion)
        imagePaths = list(paths.list_images(emotion_dir))
        for imagePath in imagePaths:
            image = cv2.imread(imagePath)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            faces = detect_faces(gray)
            for (x, y, w, h) in faces:
                face = gray[y:y+h, x:x+w]
                face_resized = cv2.resize(face, (48, 48))
                hog_features = extract_hog_features(face_resized)
                data.append(hog_features)
                labels.append(emotion)
    return np.array(data), np.array(labels)

In [8]:
# Tải dữ liệu
X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [10]:
# Xuất ra số lượng đặc trưng và số lượng mẫu
print(f"Number of features: {X_train.shape[1]}")
print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of validation samples: {X_val.shape[0]}")
print(f"Number of test samples: {X_test.shape[0]}")

Number of features: 1008
Number of training samples: 2924
Number of validation samples: 731
Number of test samples: 902


# **SVM**

In [20]:
# Sử dụng Grid Search để tìm tham số tốt nhất
param_grid = {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print("Best parameters found: ", grid_search.best_params_)

Best parameters found:  {'C': 10, 'kernel': 'rbf'}


In [21]:
# Sử dụng mô hình với tham số tốt nhất
best_svm_clf = grid_search.best_estimator_

# Đánh giá mô hình trên tập huấn luyện
y_train_pred = best_svm_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)
print("Train Classification Report:")
print(classification_report(y_train, y_train_pred))

# Đánh giá mô hình trên tập xác thực
y_val_pred = best_svm_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Đánh giá mô hình trên tập kiểm tra
y_test_pred = best_svm_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

# Lưu mô hình đã huấn luyện
dump(best_svm_clf, 'svm_emotion_recognition.joblib')

Train Accuracy: 0.9995724668661822
Train Classification Report:
              precision    recall  f1-score   support

     disgust       1.00      1.00      1.00        78
        fear       1.00      1.00      1.00       443
       happy       1.00      1.00      1.00      1372
    surprise       1.00      1.00      1.00       446

    accuracy                           1.00      2339
   macro avg       1.00      1.00      1.00      2339
weighted avg       1.00      1.00      1.00      2339

Validation Accuracy: 0.8102564102564103
Validation Classification Report:
              precision    recall  f1-score   support

     disgust       0.79      0.52      0.63        21
        fear       0.57      0.57      0.57       108
       happy       0.90      0.94      0.92       348
    surprise       0.74      0.68      0.71       108

    accuracy                           0.81       585
   macro avg       0.75      0.68      0.71       585
weighted avg       0.81      0.81      0.81    

['svm_emotion_recognition.joblib']

In [22]:
# Sử dụng Grid Search để tìm tham số tốt nhất
param_grid = {'C': [0.1, 1], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print("Best parameters found: ", grid_search.best_params_)

# Sử dụng mô hình với tham số tốt nhất
best_svm_clf = grid_search.best_estimator_

# Đánh giá mô hình trên tập huấn luyện
y_train_pred = best_svm_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)
print("Train Classification Report:")
print(classification_report(y_train, y_train_pred))

# Đánh giá mô hình trên tập xác thực
y_val_pred = best_svm_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Đánh giá mô hình trên tập kiểm tra
y_test_pred = best_svm_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

# Lưu mô hình đã huấn luyện
dump(best_svm_clf, 'svm_emotion_recognition.joblib')

Best parameters found:  {'C': 1, 'kernel': 'rbf'}
Train Accuracy: 0.9136383069687901
Train Classification Report:
              precision    recall  f1-score   support

     disgust       1.00      0.46      0.63        78
        fear       0.89      0.79      0.84       443
       happy       0.92      0.99      0.95      1372
    surprise       0.91      0.88      0.89       446

    accuracy                           0.91      2339
   macro avg       0.93      0.78      0.83      2339
weighted avg       0.91      0.91      0.91      2339

Validation Accuracy: 0.8017094017094017
Validation Classification Report:
              precision    recall  f1-score   support

     disgust       1.00      0.33      0.50        21
        fear       0.60      0.52      0.56       108
       happy       0.86      0.97      0.91       348
    surprise       0.76      0.64      0.69       108

    accuracy                           0.80       585
   macro avg       0.80      0.61      0.66       5

['svm_emotion_recognition.joblib']

In [23]:
# Sử dụng Grid Search để tìm tham số tốt nhất
param_grid = {'C': [1, 5], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print("Best parameters found: ", grid_search.best_params_)

# Sử dụng mô hình với tham số tốt nhất
best_svm_clf = grid_search.best_estimator_

# Đánh giá mô hình trên tập huấn luyện
y_train_pred = best_svm_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)
print("Train Classification Report:")
print(classification_report(y_train, y_train_pred))

# Đánh giá mô hình trên tập xác thực
y_val_pred = best_svm_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Đánh giá mô hình trên tập kiểm tra
y_test_pred = best_svm_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

# Lưu mô hình đã huấn luyện
dump(best_svm_clf, 'svm_emotion_recognition.joblib')

Best parameters found:  {'C': 5, 'kernel': 'rbf'}
Train Accuracy: 0.9987174005985464
Train Classification Report:
              precision    recall  f1-score   support

     disgust       1.00      1.00      1.00        78
        fear       1.00      1.00      1.00       443
       happy       1.00      1.00      1.00      1372
    surprise       0.99      1.00      1.00       446

    accuracy                           1.00      2339
   macro avg       1.00      1.00      1.00      2339
weighted avg       1.00      1.00      1.00      2339

Validation Accuracy: 0.811965811965812
Validation Classification Report:
              precision    recall  f1-score   support

     disgust       0.79      0.52      0.63        21
        fear       0.57      0.58      0.58       108
       happy       0.90      0.95      0.92       348
    surprise       0.74      0.67      0.70       108

    accuracy                           0.81       585
   macro avg       0.75      0.68      0.71       58

['svm_emotion_recognition.joblib']

In [25]:
# Sử dụng Grid Search để tìm tham số tốt nhất
param_grid = {'C': [0.1, 0.5], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print("Best parameters found: ", grid_search.best_params_)

# Sử dụng mô hình với tham số tốt nhất
best_svm_clf = grid_search.best_estimator_

# Đánh giá mô hình trên tập huấn luyện
y_train_pred = best_svm_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)
print("Train Classification Report:")
print(classification_report(y_train, y_train_pred))

# Đánh giá mô hình trên tập xác thực
y_val_pred = best_svm_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Đánh giá mô hình trên tập kiểm tra
y_test_pred = best_svm_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

# Lưu mô hình đã huấn luyện
dump(best_svm_clf, 'svm_emotion_recognition.joblib')

Best parameters found:  {'C': 0.5, 'kernel': 'rbf'}
Train Accuracy: 0.8388200085506626
Train Classification Report:
              precision    recall  f1-score   support

     disgust       0.00      0.00      0.00        78
        fear       0.79      0.60      0.68       443
       happy       0.85      0.99      0.91      1372
    surprise       0.84      0.76      0.80       446

    accuracy                           0.84      2339
   macro avg       0.62      0.59      0.60      2339
weighted avg       0.81      0.84      0.82      2339



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Accuracy: 0.7504273504273504
Validation Classification Report:
              precision    recall  f1-score   support

     disgust       0.00      0.00      0.00        21
        fear       0.51      0.36      0.42       108
       happy       0.80      0.97      0.88       348
    surprise       0.72      0.58      0.64       108

    accuracy                           0.75       585
   macro avg       0.51      0.48      0.49       585
weighted avg       0.70      0.75      0.72       585



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Accuracy: 0.7682926829268293
Test Classification Report:
              precision    recall  f1-score   support

     disgust       0.00      0.00      0.00        33
        fear       0.67      0.46      0.54       189
       happy       0.80      0.96      0.87       510
    surprise       0.74      0.68      0.71       170

    accuracy                           0.77       902
   macro avg       0.55      0.53      0.53       902
weighted avg       0.73      0.77      0.74       902



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['svm_emotion_recognition.joblib']

In [24]:
# Sử dụng Grid Search để tìm tham số tốt nhất
param_grid = {'C': [1, 1.5], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print("Best parameters found: ", grid_search.best_params_)

# Sử dụng mô hình với tham số tốt nhất
best_svm_clf = grid_search.best_estimator_

# Đánh giá mô hình trên tập huấn luyện
y_train_pred = best_svm_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)
print("Train Classification Report:")
print(classification_report(y_train, y_train_pred))

# Đánh giá mô hình trên tập xác thực
y_val_pred = best_svm_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Đánh giá mô hình trên tập kiểm tra
y_test_pred = best_svm_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

# Lưu mô hình đã huấn luyện
dump(best_svm_clf, 'svm_emotion_recognition.joblib')

Best parameters found:  {'C': 1.5, 'kernel': 'rbf'}
Train Accuracy: 0.9521162890123984
Train Classification Report:
              precision    recall  f1-score   support

     disgust       1.00      0.72      0.84        78
        fear       0.94      0.90      0.92       443
       happy       0.96      0.99      0.97      1372
    surprise       0.94      0.92      0.93       446

    accuracy                           0.95      2339
   macro avg       0.96      0.88      0.92      2339
weighted avg       0.95      0.95      0.95      2339

Validation Accuracy: 0.8102564102564103
Validation Classification Report:
              precision    recall  f1-score   support

     disgust       0.89      0.38      0.53        21
        fear       0.59      0.56      0.57       108
       happy       0.88      0.96      0.92       348
    surprise       0.75      0.67      0.71       108

    accuracy                           0.81       585
   macro avg       0.78      0.64      0.68      

['svm_emotion_recognition.joblib']

In [11]:
# Sử dụng Grid Search để tìm tham số tốt nhất
param_grid = {'C': [0.1, 1], 'kernel': ['linear', 'rbf']}
grid_search = GridSearchCV(SVC(), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# In ra tham số tốt nhất
print("Best parameters found: ", grid_search.best_params_)

# Sử dụng mô hình với tham số tốt nhất
best_svm_clf = grid_search.best_estimator_

# Đánh giá mô hình trên tập huấn luyện
y_train_pred = best_svm_clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)
print("Train Classification Report:")
print(classification_report(y_train, y_train_pred))

# Đánh giá mô hình trên tập xác thực
y_val_pred = best_svm_clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Đánh giá mô hình trên tập kiểm tra
y_test_pred = best_svm_clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

# Lưu mô hình đã huấn luyện
dump(best_svm_clf, 'svm_emotion_recognition.joblib')

Best parameters found:  {'C': 1, 'kernel': 'rbf'}
Train Accuracy: 0.9131326949384405
Train Classification Report:
              precision    recall  f1-score   support

     disgust       1.00      0.52      0.68        99
        fear       0.88      0.80      0.84       551
       happy       0.92      0.99      0.95      1720
    surprise       0.90      0.87      0.88       554

    accuracy                           0.91      2924
   macro avg       0.93      0.79      0.84      2924
weighted avg       0.91      0.91      0.91      2924

Validation Accuracy: 0.801641586867305
Validation Classification Report:
              precision    recall  f1-score   support

     disgust       1.00      0.30      0.46        30
        fear       0.73      0.58      0.65       161
       happy       0.85      0.95      0.90       400
    surprise       0.72      0.72      0.72       140

    accuracy                           0.80       731
   macro avg       0.82      0.64      0.68       73

['svm_emotion_recognition.joblib']

# **KNN**

In [ ]:
#Sử dụng Grid Search để tìm tham số tốt nhất
param_grid = {'n_neighbors': np.arange(1, 25), 'weights': ['uniform', 'distance'], 'metric' : ['manhattan', 'euclidean', 'cosine', 'minkowski']}
knn = KNeighborsClassifier()
knn_gscv = GridSearchCV(knn, param_grid, cv=5)
knn_gscv.fit(X_train, y_train)

#In ra bộ tham số tốt nhất
print("Best parameters found: ", knn_gscv.best_params_)

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=3, weights = 'distance', metric = 'manhattan')
knn_classifier.fit(X_train, y_train)

y_pred_test = knn_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_test)
print("Accuracy:", accuracy)

In [ ]:
# Sử dụng mô hình với tham số tốt nhất
knn_best = knn_gscv.best_estimator_

# Đánh giá mô hình trên tập xác thực
y_val_pred = knn_best.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_accuracy)
print("Validation Classification Report:")
print(classification_report(y_val, y_val_pred))

# Đánh giá mô hình trên tập kiểm tra
y_test_pred = knn_best.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_accuracy)
print("Test Classification Report:")
print(classification_report(y_test, y_test_pred))

In [ ]:
# Lưu mô hình đã huấn luyện
dump(knn_best, 'knn_emotion_recognition.joblib')

# **DEMO**

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from imutils import paths
from joblib import load

# Hàm phát hiện khuôn mặt
def detect_faces(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Hàm trích xuất đặc trưng HOG từ khuôn mặt
def extract_hog_features(image):
    fd = hog(image, orientations=7, pixels_per_cell=(8, 8), cells_per_block=(4, 4), block_norm='L2-Hys')
    return fd

# Hàm để dự đoán và lưu kết quả
def load_and_predict(imagePath, model, output_dir):
    image = cv2.imread(imagePath)
    if image is None:
        print(f"Không thể đọc ảnh từ đường dẫn: {imagePath}")
        return None, None
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detect_faces(gray)
    if len(faces) == 0:
        print(f"No faces detected in image: {imagePath}")
        return None, None
    
    prediction = None
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face_resized = cv2.resize(face, (48, 48))
        hog_features = extract_hog_features(face_resized)
        hog_features = hog_features.reshape(1, -1)
        prediction = model.predict(hog_features)[0]
        
        # Thêm nhãn dự đoán vào ảnh
        cv2.putText(image, f"Predicted: {prediction}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
        
    # Xác định đường dẫn lưu ảnh đầu ra
    output_path = os.path.join(output_dir, f"{os.path.basename(imagePath).split('.')[0]}_pred_{prediction}.jpg")
    
    # Lưu ảnh với nhãn dự đoán
    cv2.imwrite(output_path, image)
    
    return prediction, output_path

**SVM**

In [ ]:
# Tải mô hình đã huấn luyện
best_svm_clf = load('svm_emotion_recognition.joblib')

# Đường dẫn tới thư mục ảnh ngoài để dự đoán
external_image_dir = '/kaggle/input/cs231/test'
output_dir = '/kaggle/working/predictions'

# Tạo thư mục lưu kết quả nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Duyệt qua từng ảnh, dự đoán và lưu kết quả
predictions = []
no_face_detected = []
for filename in os.listdir(external_image_dir):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        imagePath = os.path.join(external_image_dir, filename)
        prediction, output_path = load_and_predict(imagePath, best_svm_clf, output_dir)
        if prediction is not None:
            predictions.append((imagePath, prediction, output_path))
            print(f'Image: {filename}, Predicted Label: {prediction}, Saved As: {output_path}')
        else:
            no_face_detected.append(imagePath)

# Lưu danh sách các dự đoán và đường dẫn ảnh
predictions, no_face_detected

In [ ]:
index = 0
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 1
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 2
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 3
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 4
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

**KNN**

In [ ]:
# Tải mô hình đã huấn luyện
knn_best = load('knn_emotion_recognition.joblib')

# Đường dẫn tới thư mục ảnh ngoài để dự đoán
external_image_dir = '/kaggle/input/cs231/test'
output_dir = '/kaggle/working/predictions'

# Tạo thư mục lưu kết quả nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Duyệt qua từng ảnh, dự đoán và lưu kết quả
predictions = []
no_face_detected = []
for filename in os.listdir(external_image_dir):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        imagePath = os.path.join(external_image_dir, filename)
        prediction, output_path = load_and_predict(imagePath, best_svm_clf, output_dir)
        if prediction is not None:
            predictions.append((imagePath, prediction, output_path))
            print(f'Image: {filename}, Predicted Label: {prediction}, Saved As: {output_path}')
        else:
            no_face_detected.append(imagePath)

# Lưu danh sách các dự đoán và đường dẫn ảnh
predictions, no_face_detected

In [ ]:
index = 0
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 1
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 2
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 3
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 4
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

# **DEMO ẢNH TỰ CHỤP**

In [ ]:
# Đường dẫn tới thư mục ảnh ngoài để dự đoán
external_image_dir = '/kaggle/input/data2testmodelcs231/test_cs231'
output_dir = '/kaggle/working/predictions'

# Tạo thư mục lưu kết quả nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Duyệt qua từng ảnh, dự đoán và lưu kết quả
predictions = []
no_face_detected = []
for filename in os.listdir(external_image_dir):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        imagePath = os.path.join(external_image_dir, filename)
        prediction, output_path = load_and_predict(imagePath, best_svm_clf, output_dir)
        if prediction is not None:
            predictions.append((imagePath, prediction, output_path))
            print(f'Image: {filename}, Predicted Label: {prediction}, Saved As: {output_path}')
        else:
            no_face_detected.append(imagePath)

# Lưu danh sách các dự đoán và đường dẫn ảnh
predictions, no_face_detected

In [ ]:
index = 0
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 1
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")

In [ ]:
index = 2
if index < len(predictions):
    imagePath, prediction, output_path = predictions[index]
    image = cv2.imread(output_path)
    if image is None:
        print(f"Failed to read image: {output_path}")
    else:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(5, 5))
        plt.imshow(image_rgb)
        plt.title(f"Predicted: {prediction}")
        plt.axis('off')
        plt.show()
else:
    print("No face detected in this image.")